# 01 - Airyjevi funkciji
<p style="font-weight: 600; text-align: center;">
Matematično-fizikalni praktikum, oktober 2023 <br>
Luka Skeledžija, 28201079
</p>

<style>
.MJXc-display{
    display: inline-block !important;
    width: 100%;
}
@media print {
    .pagebreak { page-break-before: always; } /* page-break-after works, as well */
}

img{
    width: 100%;
    max-width: 600px !important;
    margin: auto !important;
}

body {
    overflow: hidden;
    max-width: 600px;
    margin: auto;
}

::-webkit-scrollbar {
  width: 0px;
}

table{
    width: 100%;
}

h1 {
    text-transform: uppercase;
    text-align: center;
    background: #222222;
    color: white;
    padding: 8px;
}

blockquote {
    margin-left: 0em!important;
    margin-right: 0em!important;

}

.jp-RenderedHTMLCommon pre, .jp-RenderedHTMLCommon code {

    background-color: var(--jp-layout-color2)!important;
}

.jp-RenderedHTMLCommon pre{
    margin: 0.5em 0em!important;
    padding: 0em 1.5em!important;
}





</style>


---



## Uvod

Airyjevi funkciji $\text{Ai}$ in $\text{Bi}$ se v fiziki pojavljata predvsem v optiki in kvantni mehaniki. Definirani sta kot neodvisni rešitvi enačbe

$$   y''(x) -xy(x) = 0, $$ 

ki je znana kot Airyjeva oz. Stokesova enačba. To je najenostavnejša linearna diferencialna enačba drugega reda z obratno točko (tj. točko, kjer se značaj rešitev spremeni iz oscilatornega v eksponentni). Neodvisni rešitvi enačbe sta predstavljivi v integralski obliki kot

$$   \text{Ai}(x) = \frac{1}{\pi} \int_0^\infty \cos (t^3/3 + x t) \,\mathrm{d} t \>,\quad $$ 

$$   \text{Bi}(x) = \frac{1}{\pi} \int_0^\infty \left[ \mathrm{e}^{-t^3/3 + x t}
  + \sin (t^3/3 + x t) \right] \,\mathrm{d} t \>. $$

## Naloga

> Z uporabo kombinacije Maclaurinove vrste in asimptotskega razvoja poišči čim učinkovitejši postopek za izračun
vrednosti Airyjevih funkcij $\text{Ai}$ in $\text{Bi}$ na vsej realni osi
z **absolutno** napako, manjšo od $10^{-10}$. Enako naredi tudi z **relativno** napako in ugotovi,
ali je tudi pri le-tej dosegljiva natančnost, manjša od $10^{-10}$.
Pri oceni napak si pomagaj s programi, ki znajo računati s poljubno
natančnostjo.


## Ubran pristop in rešitev problema

V okviru tega poročila bomo izdelali Pythonski paket `fastAiry.py`, ki ga bomo lahko uporabili za učinkovit izračun Airyjevih funcij $\text{Ai}$ in $\text{Bi}$. Funkciji sta definirani na celotni realni osi, njuna izračunana vrednost pa se mora nahajati v okviru neke poljubne absolutne oz. relativne napake. Paket bomo intenzivno testirali na primeru, ko je zahtevana relativna in absolutna napaka manjša od $ 10^{-10}$.

**Lastnosti Airyjevih funkcij**

Funkciji $\text{Ai}$ in $\text{Bi}$ sta definirani na celotni realni osi. Za vrednosti $ x \in \mathbb{R}^-$ sta obe funkciji oscilatorni in omejeni. Za $ x \in \mathbb{R}^+ \cup \{0\}$ pa je funkcija $\text{Ai}$ omejena, medtem ko $\text{Bi}$ narašča čez vse meje. 


![Airy Functions](./media/airyf.png)
> Graf funkcij $\text{Ai}$ in $\text{Bi}$

**Matematični pristop in potencialne težave**

Za izračun vrednosti funkcij integralov ne bomo neposredno izvrednotili, temveč bomo funkcije razvili v Maclaurinovo in asimptotsko vrsto. Pri tem bomo pazili, da formule implementiramo rekurzivno in tako čim bolj zmanjšamo število potrebnih računskih operacij. Na grobo ocenimo, da z uporabo rekurzivnih formul zmanjšamo časovno zahtevnost programa iz $O(n^2)$ na $O(n)$, kar v računalniškem lingu razume kot "iz zelo počasi v precej hitro".

Za natančnost bomo poskrbeli z ustreznim izborom števila členov v Maclaurinovi oz. asimptotski vrsti. Ker ena vrsta podaja dober približek za zelo majhne vrednosti $x$, druga pa za zelo velike, bomo vrste v točki $x_0$ zlepili. Točko $x_0$ bomo določili na podlagi števila členov, ki jih potrebuja ena in druga vrsta za doseganje tolerirane napake.  

Potencialne težave bodo (in tudi so) predstavljale predvsem: omejitve podatkovnega tipa `float`, iskanje zanesljive referenčne vrednosti za oceno napak in iskanje minimuma napake glede na število členov asimptotske vrste.





## Referenčna funkcija

Za primerjavo naših vrst z referenčno vrednostjo lahko uporabimo funkciji `mpmath.airyai()` in `mpmath.airybi()` iz Pythonskega paketa `mpmath`. Ker bo to naša osnova za nadaljevanje, je smiselno preveriti, če ti dve funkciji dejansko delujeta pravilno oz. dovolj natančno glede na definicijo Airyjevih funkcij. 

Z uporabo programa `Wolfram Mathematica` numerično integriramo definiciji funkcij $ \text{Ai} $ in $ \text{Bi} $ v nekaj testnih točkah s sledečimi nastavitvami:

```

x = 20;

integrandA[t_] := Cos[t^3/3 + x*t];
NIntegrate[integrandA[t], {t, 0, Infinity}, PrecisionGoal -> 30, AccuracyGoal -> 30, WorkingPrecision -> 50];

integrandB[t_] := Exp[-t^3/3 + x*t] + Sin[t^3/3 + x*t];
NIntegrate[integrandB[t], {t, 0, Infinity}, PrecisionGoal -> 30, AccuracyGoal -> 30, WorkingPrecision -> 50];

```

Rezultate primarjamo s funkcijama `mpmath.airyai()` in `mpmath.airybi()`, ki ju nastavimo na 50 decimalk.

## Maclaurinov približek

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp

# Maclaurent series for the Airy functions

def f_mac(x, n):
    z = 1/3
    k = np.arange(1, n+1, 1)
    k_series = 1 * np.cumprod(3 * x**3 / ( (3*k+1) * (3*k+2) * (3*k+3) ) * (z + k) )
    return np.sum(k_series) + 1

def g_mac(x, n):
    z = 2/3
    k = np.arange(1, n+1, 1)
    k_series = x * np.cumprod(3 * x**3 / ( (3*k+2) * (3*k+3) * (3*k+4) ) * (z + k) )
    return np.sum(k_series) + x

def airy_mac(x_range, n=10):
    alpha = 0.355028053887817239
    beta = 0.258819403792806798
    A_i = np.zeros(len(x_range), dtype=float)
    B_i = np.zeros(len(x_range), dtype=float)
    for i, x in enumerate(x_range):
        f =  f_mac(x, n)
        g =  g_mac(x, n)
        np.put(A_i, i, alpha * f - beta * g)
        np.put(B_i, i, np.sqrt(3) * (alpha * f + beta * g))
    return [A_i, B_i]

lim_x = 1
lim_y = 1.2
x_range = np.linspace(-2, 2, 500)
A_i, B_i = airy_mac(x_range)
A_i_ref, _, B_i_ref, _ = sp.special.airy(x_range)
plt.plot(x_range, A_i, label='A_i')
plt.plot(x_range, B_i, label='B_i')
plt.plot(x_range, A_i_ref, label='A_i_ref')
plt.plot(x_range, B_i_ref, label='B_i_ref')
plt.grid()
#plt.ylim(-lim, lim)
plt.legend()
plt.show()

